# Melon 주간 Top 100 구하기

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request

In [7]:
import requests
url ='https://www.melon.com/chart/week/index.htm'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
req = requests.get(url, headers = headers)
html = req.text

In [8]:
soup = BeautifulSoup(html,'html.parser')

### 차트 기간

In [12]:
soup.select_one('.yyyymmdd').get_text()

'\r\n\t\t\t\t\t\t\t2020.11.09 ~ 2020.11.15\r\n\t\t\t\t\t\t'

In [13]:
date = soup.select_one('.yyyymmdd').get_text().strip()
date

'2020.11.09 ~ 2020.11.15'

In [14]:
start_day = date[:10]
end_day = date[13:]
start_day, end_day

('2020.11.09', '2020.11.15')

### 곡 리스트 찾기

In [27]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

In [51]:
rank = int(trs[0].select_one('.rank').string)
print(rank)

1


In [50]:
for i in [0, 1, 2, 19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    print(diff)

0
1
-1
-100


In [115]:
info = trs[0].select_one('.ellipsis.rank01')
title = info.find('a').string
info = trs[0].select_one('.ellipsis.rank02')
singer = info.find('a').string
info = trs[0].select_one('.ellipsis.rank03')
album = info.find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

#### DataFrame으로 만들기

In [119]:
ranks = []; diffs = []; 
titles = []; singers = []; albums = []
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [120]:
for tr in trs:
    rank = int(tr.select_one('.rank').string)
    info = tr.select_one('.ellipsis.rank01')
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string
    
    entry = tr.select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    ranks.append(rank)
    diffs.append(diff)    
    titles.append(title)        
    singers.append(singer)
    albums.append(album)

In [118]:
Melon_top_100 = pd.DataFrame({
                    'rank' : ranks,
                    'diff' : diffs,
                    'title' : titles,
                    'singer' : singers,
                    'album' : albums
})
Melon_top_100 = Melon_top_100.set_index('rank')
print(f'기간 : {start_day} ~ {end_day}')
Melon_top_100

기간 : 2020.11.09 ~ 2020.11.15


,diff,title,singer,album
rank,,,,
1,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,1,Dynamite,방탄소년단,Dynamite (DayTime Version)
3,-1,Dynamite,방탄소년단,Dynamite (DayTime Version)
4,1,Dynamite,방탄소년단,Dynamite (DayTime Version)
5,-1,Dynamite,방탄소년단,Dynamite (DayTime Version)
...,...,...,...,...
96,-4,Dynamite,방탄소년단,Dynamite (DayTime Version)
97,-100,Dynamite,방탄소년단,Dynamite (DayTime Version)
98,-3,Dynamite,방탄소년단,Dynamite (DayTime Version)
